# FROM Aula 72 a 75 - Deep Learning - Prof. Tiago Santana-20200320T211121Z-001

In [18]:
import numpy as np
import pandas as pd

import tensorflow as tf
from glob import glob
from tensorflow.keras.applications import vgg16, ResNet152V2, NASNetLarge
from tensorflow.keras.preprocessing import image
import os

In [2]:
with tf.compat.v1.Session() as sess:
  devices = sess.list_devices()
  print(devices)

[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 3274866592812707203), _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 6676091371, 9927651758008242774)]


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print(tf.test.gpu_device_name())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7830881997837045341
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6676091371
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8997750268562077725
physical_device_desc: "device: 0, name: GeForce GTX 1070 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
/device:GPU:0


In [14]:
current_path = os.getcwd()
image_path = current_path+"\\only_faces\\"
image_destination = image_path+"balanced\\"
DATASET_FOLDER = '\\tropical_fruits1400'

vgg16_model = vgg16.VGG16(weights='imagenet', include_top=False, pooling='max')
Resnet_model = ResNet152()
NAS_model = NASNetLarge()

print(tf.version.GIT_VERSION, tf.version.VERSION)

v2.1.0-rc2-17-ge5bf8de410 2.1.0


# Extrair features das imagens

In [26]:
def describe_im_vgg16(vgg16_model, path):
    img = image.load_img(path, target_size=(224, 224))
    #print(f"img = {type(img)}")
    x = image.img_to_array(img)
    print(f"x tem {len(x)}")
    x = np.expand_dims(x, axis=0)
    x = vgg16.preprocess_input(x)
    print(f"x = {type(x)}")
    print(f"x preprocess_input tem {len(x)}")
    #print(x)
    features = vgg16_model.predict(x)#.reshape(-1)
    #print(f"features = {type(features)}")
    #print(f"features = {features}")
    #print(f"features.tolist = {features.tolist()}")
    print(f"features.tolist() tem {len(features.tolist())}")
    
    return features.tolist()

In [27]:
col_names = ['im_path'] + ['f%03d' % (i) for i in range(1, 513)] + ['class']
df = pd.DataFrame(columns=col_names)

# n_classes = 3
# for c in range(n_classes):
n_classes = ["without_mask","with_mask","mask_weared_incorrect"]
for c in n_classes:
    #print(f"c = {c}")
    for p in glob('%s/%s/*.png' % (image_destination, c)):
        #print(f"p = {p}")
        features = describe_im_vgg16(vgg16_features, p)
        print(f"describe_im_vgg16 = {features}")
        df.loc[len(df)] = [p] + features + [c]
        print(p)

x tem 224
x = <class 'numpy.ndarray'>
x preprocess_input tem 1


LookupError: No gradient defined for operation 'IteratorGetNext' (op type: IteratorGetNext)

In [ ]:
df.to_csv('TCC_Wesley.csv', index=None)

# Criar bases de treino e teste

In [ ]:
df = pd.read_csv('TCC_Wesley.csv')

df.head()

In [ ]:
X, y = df.drop(['im_path', 'class'], axis=1), df['class'].values

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Treinar um modelo (Gradient Boosting)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)

In [ ]:
gb.score(X_test, y_test)

# Classificar uma nova imagem

In [ ]:
features_im = describe_im_vgg16(vgg16_features, 'melancia.jpg')

probs = gb.predict_proba(np.array(features_im).reshape(1, -1))

probs

In [ ]:
np.max(probs), np.argmax(probs)